데이터 통합 및 전처리 (Preprocessing)

1-1. 데이터 읽기 및 통합 함수 정의

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# '지역구' 인덱스 통일화를 위한 함수 (예: '강남구' -> '강남')
def normalize_gu_name(df, col_name='지역구'):
    """데이터프레임 내 '지역구' 컬럼 이름을 통일된 형태로 정리합니다."""
    if col_name in df.columns:
        # '구' 제거 및 공백 정리
        df[col_name] = df[col_name].astype(str).str.replace('구', '').str.strip() 
        return df
    return df

# 통합 데이터프레임 초기화 (X 요소와 C 기준을 위한 초기화 포함)
df_combined = pd.DataFrame()

# ==============================================================================
# 1-1. C 및 X 요소 통합 (이전 단계 코드 재구성)
# ==============================================================================
# C (기준 지표): X03_전처리_범죄발생_건수_20231231기준.csv (성범죄 합계)
df_c = pd.read_csv('X03_전처리_범죄발생_건수_20231231기준.csv')
df_c = normalize_gu_name(df_c)
df_c.set_index('지역구', inplace=True)
df_combined = df_c[['합계']].rename(columns={'합계': 'C_성범죄_합계'})

# X 요소 파일 및 추출 컬럼 매핑
x_files_map = {
    'X_5대범죄_발생': ('X01_전처리_5대범죄_발생검거_누적건수.csv', '총_누적_발생_건수'),
    'X_112신고출동': ('X02_전처리_112신고출동_연간건수_2020_2024.csv', '2024'),
    'X_유흥주점': ('X09_전처리_유흥주점_수.csv', '지역구별 주점 수'),
    'X_여성독거노인': ('X13_전처리_여성독거노인_현황_2020_2024.csv', '2024'),
}

for col_name, (file_name, data_col) in x_files_map.items():
    try:
        df_temp = pd.read_csv(file_name)
        if '지역구' in df_temp.columns:
            df_temp = normalize_gu_name(df_temp)
            df_temp.set_index('지역구', inplace=True)
        
        if data_col in df_temp.columns:
            df_temp = df_temp[[data_col]]
            df_temp.columns = [col_name]
            df_combined = df_combined.join(df_temp, how='left')
        else:
            print(f"[X 통합 경고] {file_name}: 컬럼 '{data_col}' 누락.")
    except Exception as e:
        print(f"[X 통합 오류] {file_name}: {e}")

# ==============================================================================
# 1-2. Y 요소 통합 (새로운 파일)
# ==============================================================================
y_files_map = {
    'Y_안심벨_CCTV': ('Y06_전처리_CCTV_설치수_20230421기준..csv', '안심벨 설치수'),
    'Y_귀갓길CCTV': ('Y06_전처리_서울시 안심귀갓길 안전시설물_CCTV.csv', 'CCTV 설치수'),
    'Y_지구대파출소': ('Y14_전처리_지구대파출소_설치수_20241231기준.csv', '지역구별 지구대/파출소 수'),
    'Y_치안센터': ('Y14_전처리_치안센터_설치수_20250630기준.csv', '지역구별 치안센터 수'),
    'Y_경찰관': ('Y15_전처리_경찰관_인원수_2020_2024.csv', '2024'),
}

print("\n[Y 요소 통합 시작]")

# Y06_전처리_안심벨_설치수_20230421기준.csv는 복잡하여 수동 처리 (안심귀갓길 시설물 합산)
try:
    df_y06_detail = pd.read_csv('Y06_전처리_안심벨_설치수_20230421기준.csv')
    df_y06_detail.rename(columns={'구': '지역구'}, inplace=True)
    df_y06_detail = normalize_gu_name(df_y06_detail)
    
    # 구별 '안심벨 설치수' 합계
    df_y_bell_sum = df_y06_detail.groupby('지역구')['안심벨 설치수'].sum().to_frame('Y_안심벨_합계')
    df_combined = df_combined.join(df_y_bell_sum, how='left')
    print("  + Y_안심벨_합계 통합 성공.")
except Exception as e:
    print(f"[Y 통합 오류] Y06_전처리_안심벨_설치수_20230421기준.csv 처리 중 예외 발생: {e}")

# 나머지 Y 파일 통합
for col_name, (file_name, data_col) in y_files_map.items():
    try:
        df_temp = pd.read_csv(file_name)
        if '지역구' in df_temp.columns:
            df_temp = normalize_gu_name(df_temp)
            df_temp.set_index('지역구', inplace=True)
        
        if data_col in df_temp.columns:
            df_temp = df_temp[[data_col]]
            df_temp.columns = [col_name]
            df_combined = df_combined.join(df_temp, how='left')
            print(f"  + {col_name} 통합 성공.")
        else:
            print(f"[Y 통합 경고] {file_name}: 컬럼 '{data_col}' 누락.")
    except Exception as e:
        print(f"[Y 통합 오류] {file_name}: {e}")

# ==============================================================================
# 1-3. 최종 데이터 정리 및 정규화 (Min-Max Scaling)
# ==============================================================================
# 결측치 처리 (통합되지 못한 지역구의 값은 0으로 간주)
df_combined.fillna(0, inplace=True)

# 정규화
for col in df_combined.columns:
    col_min = df_combined[col].min()
    col_max = df_combined[col].max()
    
    if col_max == col_min:
        df_combined[col + '_norm'] = 0.5 # 데이터가 모두 같으면 0.5로 처리
    else:
        df_combined[col + '_norm'] = (df_combined[col] - col_min) / (col_max - col_min)

print("\n" + "="*70)
print("✅ 1단계: 통합 및 정규화 완료 (Min-Max Scaling)")
print(df_combined.head())
print(f"\n총 지역구 수: {len(df_combined)}개, 총 변수 수: {len(df_combined.columns)}개")
print("="*70)


[Y 요소 통합 시작]
  + Y_안심벨_합계 통합 성공.
  + Y_안심벨_CCTV 통합 성공.
  + Y_귀갓길CCTV 통합 성공.
  + Y_지구대파출소 통합 성공.
  + Y_치안센터 통합 성공.
  + Y_경찰관 통합 성공.

✅ 1단계: 통합 및 정규화 완료 (Min-Max Scaling)
     C_성범죄_합계  X_5대범죄_발생  X_112신고출동  X_유흥주점  X_여성독거노인  Y_안심벨_합계  Y_안심벨_CCTV  \
지역구                                                                           
강남       1099      81473   103837.0     165     16359        89          89   
강동        294      44997    88670.0      98     14763        46          46   
강북        261      35873    73199.0      69     14192        77          77   
강서        556      52290   115142.0     126     26350        34          34   
관악        636      58521   112242.0     190     15566        49          49   

     Y_귀갓길CCTV  Y_지구대파출소  Y_치안센터  ...  X_5대범죄_발생_norm  X_112신고출동_norm  \
지역구                               ...                                   
강남         122        14     2.0  ...        1.000000        0.802456   
강동          48         9     0.0  ...        0.360597    

In [9]:
# ==============================================================================
# 2단계: 상관분석 및 가중치 산출 (Statistical Analysis)
# ==============================================================================

# 기준 변수 C (성범죄 합계)
C = df_combined['C_성범죄_합계']
# 분석 대상 (정규화된 X와 Y 요소들)
norm_cols = [col for col in df_combined.columns if col.endswith('_norm') and col != 'C_성범죄_합계_norm']

correlations = {}
selected_x_weights = {} 
selected_y_weights = {} 

# 상관분석 및 가중치 산출
for col_norm in norm_cols:
    data_norm = df_combined[col_norm]
    
    # 피어슨 상관계수 (r)와 p-value 산출
    r, p = pearsonr(data_norm, C)
    
    # 신뢰도 95% 수준 (p-value < 0.05)에서 유의미한 요소만 선정
    if p < 0.05:
        W = np.abs(r) # 가중치 W는 상관계수 절댓값
        
        correlations[col_norm] = {'r': r, 'p': p, 'W': W, '유의미': 'O'}
        
        # X와 Y로 구분하여 최종 가중치 리스트에 저장
        if col_norm.startswith('X_'):
            selected_x_weights[col_norm] = W
        elif col_norm.startswith('Y_'):
            selected_y_weights[col_norm] = W
    else:
        correlations[col_norm] = {'r': r, 'p': p, 'W': 0.0, '유의미': 'X'}

df_corr = pd.DataFrame(correlations).T.sort_values(by=['유의미', 'W'], ascending=[False, False])

print("\n" + "="*70)
print("✅ 2단계: 상관분석 결과 및 가중치 (p < 0.05만 유의미)")
print(df_corr)
print("\n=> 최종 선정된 X 요소 (위험/감점) 및 가중치:", selected_x_weights)
print("=> 최종 선정된 Y 요소 (안전/가점) 및 가중치:", selected_y_weights)
print("="*70)


✅ 2단계: 상관분석 결과 및 가중치 (p < 0.05만 유의미)
                        r         p         W 유의미
X_여성독거노인_norm    0.153774  0.463018       0.0   X
Y_지구대파출소_norm    0.258304  0.212512       0.0   X
Y_치안센터_norm      0.115814  0.581439       0.0   X
X_5대범죄_발생_norm   0.874558       0.0  0.874558   O
X_112신고출동_norm   0.580251   0.00236  0.580251   O
Y_경찰관_norm       0.542137  0.005118  0.542137   O
Y_귀갓길CCTV_norm    0.50416  0.010177   0.50416   O
Y_안심벨_합계_norm    0.460346  0.020578  0.460346   O
Y_안심벨_CCTV_norm  0.460346  0.020578  0.460346   O
X_유흥주점_norm      0.439541  0.027915  0.439541   O

=> 최종 선정된 X 요소 (위험/감점) 및 가중치: {'X_5대범죄_발생_norm': np.float64(0.8745580370131926), 'X_112신고출동_norm': np.float64(0.580250506141048), 'X_유흥주점_norm': np.float64(0.4395410179951516)}
=> 최종 선정된 Y 요소 (안전/가점) 및 가중치: {'Y_안심벨_합계_norm': np.float64(0.46034649156642893), 'Y_안심벨_CCTV_norm': np.float64(0.46034649156642893), 'Y_귀갓길CCTV_norm': np.float64(0.50415965464126), 'Y_경찰관_norm': np.float64(0.5421369938388992)}


In [6]:
!pip install tabulate

In [10]:
# ==============================================================================
# 3단계: 여성안전지수(S_gu) 산출
# ==============================================================================

S_gu = pd.Series(0.0, index=df_combined.index)

# 1. Y 요소 (안전/가점) 합산
print("\n[3단계] 안전(Y) 요소 가점 계산 중...")
for col_norm, W in selected_y_weights.items():
    print(f"  + {col_norm}: 가중치 {W:.4f}")
    S_gu += df_combined[col_norm] * W

# 2. X 요소 (위험/감점) 감산
print("[3단계] 위험(X) 요소 감점 계산 중...")
for col_norm, W in selected_x_weights.items():
    print(f"  - {col_norm}: 가중치 {W:.4f}")
    S_gu -= df_combined[col_norm] * W

# 3. 최종 지수 스케일링 (0~100)
S_gu_min = S_gu.min()
S_gu_max = S_gu.max()

# 0으로 나누는 오류 방지
if S_gu_max != S_gu_min:
    df_combined['여성안전지수_Sgu'] = ((S_gu - S_gu_min) / (S_gu_max - S_gu_min)) * 100
else:
    df_combined['여성안전지수_Sgu'] = 50.0

df_combined['여성안전지수_Sgu'] = df_combined['여성안전지수_Sgu'].round(2)

print("\n" + "="*70)
print("✅ 3단계: 최종 지역구별 여성안전지수(Sgu) 결과 (0-100점)")
print("=> 지수가 높을수록 안전한 지역입니다.")
print(df_combined[['C_성범죄_합계', '여성안전지수_Sgu']].sort_values(by='여성안전지수_Sgu', ascending=False))
print("="*70)


[3단계] 안전(Y) 요소 가점 계산 중...
  + Y_안심벨_합계_norm: 가중치 0.4603
  + Y_안심벨_CCTV_norm: 가중치 0.4603
  + Y_귀갓길CCTV_norm: 가중치 0.5042
  + Y_경찰관_norm: 가중치 0.5421
[3단계] 위험(X) 요소 감점 계산 중...
  - X_5대범죄_발생_norm: 가중치 0.8746
  - X_112신고출동_norm: 가중치 0.5803
  - X_유흥주점_norm: 가중치 0.4395

✅ 3단계: 최종 지역구별 여성안전지수(Sgu) 결과 (0-100점)
=> 지수가 높을수록 안전한 지역입니다.
     C_성범죄_합계  여성안전지수_Sgu
지역구                      
성북        312      100.00
강북        261       90.36
동작        355       72.81
용산        371       51.75
노원        359       48.50
도봉        183       47.59
중랑        289       47.12
서대문       266       45.07
서초        629       44.09
강남       1099       37.58
동대문       289       36.94
성동        251       36.52
관악        636       29.05
강동        294       28.23
금천        210       25.94
종로        362       24.64
광진        412       23.75
양천        242       17.34
로         328       11.73
송파        557       11.23
중         324        7.73
은평        309        4.47
영등포       503        0.52
강서        556        0.0

In [13]:
# df_combined에 이미 '여성안전지수_Sgu' (0~100) 컬럼이 있다고 가정합니다.

# 1. 원하는 범위 설정
MIN_SCORE_ORIGINAL = 0.0
MAX_SCORE_ORIGINAL = 100.0

MIN_SCORE_TARGET = 35.0  # 논문과 같이 최저점을 35로 설정
MAX_SCORE_TARGET = 100.0 # 최고점을 100으로 설정 (변화 없음)

# 2. 선형 변환(재정규화) 공식 적용
# New_Value = New_Min + (Old_Value - Old_Min) * (New_Max - New_Min) / (Old_Max - Old_Min)
# Old_Min = 0, Old_Max = 100이므로 공식은 다음과 같이 단순화됩니다:
# New_Value = 35 + (여성안전지수_Sgu / 100) * (100 - 35)

df_combined['여성안전지수_논문유사'] = MIN_SCORE_TARGET + \
                                      (df_combined['여성안전지수_Sgu'] / MAX_SCORE_ORIGINAL) * \
                                      (MAX_SCORE_TARGET - MIN_SCORE_TARGET)

df_combined['여성안전지수_논문유사'] = df_combined['여성안전지수_논문유사'].round(2)

print("\n" + "="*70)
print("🏆 최종 여성 안전지수 재정규화 결과 (논문 유사: 최저 35점)")
print("=> 마포구의 0점이 35점으로 변환됨.")

df_result = df_combined[['여성안전지수_Sgu', '여성안전지수_논문유사']].sort_values(by='여성안전지수_논문유사', ascending=False)
df_result['순위'] = df_result['여성안전지수_논문유사'].rank(ascending=False, method='min').astype(int)

print(df_result[['순위', '여성안전지수_Sgu', '여성안전지수_논문유사']].head(25).to_markdown(floatfmt=".2f"))
print("="*70)


🏆 최종 여성 안전지수 재정규화 결과 (논문 유사: 최저 35점)
=> 마포구의 0점이 35점으로 변환됨.
| 지역구   |   순위 |   여성안전지수_Sgu |   여성안전지수_논문유사 |
|:---------|-------:|-------------------:|------------------------:|
| 성북     |   1.00 |             100.00 |                  100.00 |
| 강북     |   2.00 |              90.36 |                   93.73 |
| 동작     |   3.00 |              72.81 |                   82.33 |
| 용산     |   4.00 |              51.75 |                   68.64 |
| 노원     |   5.00 |              48.50 |                   66.53 |
| 도봉     |   6.00 |              47.59 |                   65.93 |
| 중랑     |   7.00 |              47.12 |                   65.63 |
| 서대문   |   8.00 |              45.07 |                   64.30 |
| 서초     |   9.00 |              44.09 |                   63.66 |
| 강남     |  10.00 |              37.58 |                   59.43 |
| 동대문   |  11.00 |              36.94 |                   59.01 |
| 성동     |  12.00 |              36.52 |                   58.74 |
| 관악     |  13.00 | 